In [1]:
load("ckks.py")

In [2]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Number of levels
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 3.+3.j,  3.+1.j, -3.-2.j,  3.+0.j]),
 array([-2.+2.j, -0.-2.j, -3.-3.j, -0.-2.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[12884901888 + 17997145388X^4096 - 9111001500X^8192 + 14029113011X^12288 + 4294967296X^16384 + 4167436050X^20480 + 9111001500X^24576 - 5811048877X^28672 mod(2^700),
 - 10737418240 + 6093048256X^4096 - 3037000500X^8192 + 10741887357X^12288 - 10737418240X^16384 + 9098274531X^20480 + 12148002000X^24576 + 2125015878X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.+3.00000000e+00j  3.+1.00000000e+00j -3.-2.00000000e+00j
  3.-2.18062124e-10j]
[-2.00000000e+00+2.j -4.26597609e-11-2.j -3.00000000e+00-3.j
 -2.78597482e-10-2.j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 1.+5.j  3.-1.j -6.-5.j  3.-2.j]
[-1.20000000e+01-4.79891682e-09j  2.00000000e+00-6.00000000e+00j
  3.00000000e+00+1.50000000e+01j -1.53639124e-09-6.00000000e+00j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[ 2.99523992+2.99697403e+00j  2.99781809+9.97432509e-01j
 -2.9965996 -1.99706696e+00j  2.9981091 +2.46998590e-03j]
